### Plan of action
- Exploratory Data Analysis (EDA)
- Data cleaning
- Missingness imputation
- Encoding categorical variables
- Outlier removal
- Feature engineering
- Scaling
- Cross-Validation ( Hyperparameter tuning)
- Modeling
- Deep Learning Techniques

The dataset used in this assignment is described by DeCock at http://jse.amstat.org/v19n3/decock/DataDocumentation.txt

Before starting a project, it is important to have an empty environment. This means that no python objects are saved in memory.

In [ ]:
%whos

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Importing libraries for manipulating arrays, dataframes and plotting the data. These libraries are Numpy (arrays), matplotlib and seaborn (plotting) as well as pandas (dataframes)

In [ ]:
import numpy as np # manipulation of arrays
import pandas as pd # manipulating dataframes
import matplotlib.pyplot as plt # data visualisation
plt.style.use('ggplot')
import seaborn as sns # data visualisation,it is based on plt

Reading the data using pandas.read_csv function for comma seperated (.csv) files.

In [ ]:
df_train = pd.read_csv('../input/group-assignment-ace-2020/train.csv')
df_test = pd.read_csv('../input/group-assignment-ace-2020/test.csv')

For the best perfomance and also to avoid redundancy in work, the datasets are combined such that the same transformations are applied to both datasets. The last column in the training data is removed since it is the target variable, the transformations are applied to the combined dataset.

The removed variable is assigned to a new variable (it will be needed later).


In [ ]:
# name of last column in training set
df_train.columns[-1] 

In [ ]:
y_train=df_train.iloc[:,-1].values
df_train.drop(["SalePrice"],axis=1,inplace=True)
data=pd.concat([df_train,df_test])

The SalePrice column is analysed using the density plot to understand its distribution. The seaborn library has a consice way of doing this.

In [ ]:
# density plot
ax = sns.distplot(y_train)
ax.set(xlabel = "Intervals", ylabel = "Density")
plt.show()

SalePrice is not normally distributed. There is right skewedness which means that a small number of houses have a very high price. This suggests log transformation of the variable in order to have a normal distribution.

In [ ]:
from scipy import stats
figure = plt.figure(figsize = (13,5))
plt.subplot(1,2,1)
stats.probplot(y_train, plot = plt)
plt.title('Actual SalePrice')
plt.subplot(1,2,2)
ytrain_log = np.log(y_train)
stats.probplot(ytrain_log, plot = plt)
plt.title('SalePrice after log transformation')
plt.show()

The plot above is a QQ-plot. For normally distributed data, all values lies along the diagonal across the plot as in the second plot. The few point away from the diagonal are outliers in the data.

#### Checking the data types of variables in the dataframe <br/>
This is important because most algorithms in the scikit-learn expect numerical values. Any non-numeric values are encoded into numeric variables.

In [ ]:
data.info(memory_usage='deep')

There are 43 non-numeric variables (object(43)) and 37 numeric variables (float64(11) and int64(26)).

There are also columns with a lot of missing data such as PoolQC with 10 observed values. This means that most houses do not have a swimming pool. Other variables with a large proportion of missing data are MiscFeature, and Fence.

#### Checking for proportion of missing data <br/>
The impact of missing data on quantitative research can be serious, leading to biased estimates of parameters, loss of information, decreased statistical power, increased standard errors, and weakened generalizability of findings,(Dong & Peng). http://ncbi.nlm.nih.gov/pmc/articles/PMC3701793/

In [ ]:
notNA = data.count()/1460  # diving the number of observed non-missing values by the number of rows

for i in notNA:
    if i < 0.5: # less than 50% of observed data
        print("Proportion of non-missing data %5.4f at column %d" % (i, list(notNA).index(i)+1))

It can be seen that the 5th, 71th, 72th and 73th columns have more than 50% of the data missing. Rememebr pythonis zero-indexed.

Alternatively, the is.na() finction can be used. This function operates columnwise and returns a boolean dataframe of the same dimensions as the first one. The values can be summed up and the values divided by the number of observations which is the same. as finding the mean. Multiplying the value by 100 returns the percentage of missing values. (isna().sum()/len(df))

In [ ]:
missing = data.isna().mean().round(5)*100

for i in missing:
    if i > 30: # setting threshold for missing data (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3701793/)
        print("Proportion of missing data:  %5.4f at column %d" % (i, list(missing).index(i)+1))

In [ ]:
# visualisation using a bar plot
# Finding out the columns that are missing values in the dataset

missing.sort_values(inplace=True)
missing.plot.bar(figsize=(15,9), x = 'Features', y = 'Percentage of missing data')

All the features with non-zero values have missing values. However, Fence, Alley, MiscFeature and PoolQC have the largest number of missing values  with PoolQC having the largest value. 

In [ ]:
data.columns[[6,57,72,73,74]] #python is zero-indexed

The columns above will be removed/dropped. The criterion for dropping a feature was "Any feature with more than 30% of the data as missing". It is stringent but makes analysis easier as well as more accurate predictions since a large number of values are not imputed.

In [ ]:
miss_features = missing.loc[missing>30].index
list(miss_features)

In [ ]:
data.drop(miss_features,inplace=True,axis=1)

### Encoding categorical or non-numeric variables

Encoding methods available in python are; **One Hot Encoding, Label Encoding, Ordinal Encoding,Helmert Encoding, Binary Encoding, Frequency Encoding, Mean Encoding, Weight of Evidence Encoding, Probability Ratio Encoding, Hashing Encoding, Backward Difference Encoding, Leave One Out Encoding, James-Stein Encoding, M-estimator Encoding**

https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02

https://towardsdatascience.com/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159

https://osf.io/356ed/download (A Benchmark Experiment on How to Encode Categorical Features in Predictive Modeling, Florian Pargent)

It is necessary to tell apart nominal and ordinal variables as they may require different encoding.

In [ ]:
#getting a list of non-numeric columns

num_cols = list(data._get_numeric_data().columns) # getting numeric columns
cols = list(data.columns) # all the columns

cat_cols = [item for item in cols if item not in num_cols] # use set difference list(set(cols) - set(num_cols))
cat_cols

In [ ]:
# identifying nominal, binary and ordinal variables using the dataset description
categorical = pd.DataFrame()
                           
for var in cat_cols:
    temp = [var, data[var].nunique(), (data[var].unique())]
    categorical = categorical.append(pd.Series(temp), ignore_index=True)

categorical.columns = ['feature','number of unique features','unique features']

In [ ]:
categorical
categorical.style.set_properties(subset=['unique features'], **{'width': '300px'})

Nominal features (cannot be ordered) - LotConfig, Neighborhood, Condition1, Condition2, BldgType, HouseStyle,RoofStyle, RoofMatl, Exterior1st, Exterior2nd, Foundation, Heating, GarageType, MSZoning, LandContour, Street, CentralAir, SaleType, SaleCondition, MassVnrType

MSSubType is already encoded

Possible encoders - **Target encoding**

Ordinal variables (can be ordered) - LotShape, Utilities, LandSlope, ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, HeatingQC, KitchenQual, Functional, FirePlaceQu, GarageFinish, GarageQual, GarageCond, BsmtFinType1, BsmtType2, Electrical, PavedDrive

Possible encoding - **Ordinal encoding**

OverallCond and OverallQual are ordinal variables but are already encoded

In [ ]:
nominal = ['LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 
           'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 
           'Foundation', 'Heating', 'GarageType', 'MSZoning', 'LandContour', 
           'Street', 'CentralAir', 'SaleType', 'SaleCondition', 'MasVnrType']

ordinal = ['LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 
           'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'KitchenQual', 
           'Functional', 'GarageFinish', 'GarageQual', 'GarageCond', 
           'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'PavedDrive']


Before encoding the non-numerical variables, it is important to identify features with high cardinality (those exhibiting a large number of levels). These have an non-productive effect in the case that decision trees are used in supervised learning. The number of splits done increases the computing power which may not be available.

In [ ]:
# checking for cardinality (feature with a high number of levels)
sum(categorical['number of unique features']>=8)

In [ ]:
# seperating categorical features
catColumns = data.select_dtypes(include=['object']).copy()
# print(catColumns.isnull().values.sum())
print(catColumns.isnull().mean())

In [ ]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder

# for nomimal variables  https://medium.com/analytics-vidhya/types-of-categorical-data-encoding-schemes-a5bbeb4ba02b

# create an object of the OrdinalEncoding
ce_ordinal = ce.OrdinalEncoder(cols=nominal)
# fit and transform and you will get the encoded data
catColumns = ce_ordinal.fit_transform(catColumns)

 For ordinal variables, ordinal order  was assigned through through dictionaries.
 Variables considered are; LandSlope, ExtrQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, HeatingQC, KitchenQual, Functional, GarageFinish, GarageQual, GarageCond, SaleType. SaleCondition

In [ ]:
LandSlope_dict = {'Gtl': 1,
                      'Med': 2,
                      'Sev': 3}
catColumns['LandSlope'] = catColumns.LandSlope.map(LandSlope_dict)

In [ ]:
ExterQual_dict = {'Gd': 3,
                      'TA': 2,
                      'Ex': 4,
                     'Fa':1
                    }
catColumns['ExterQual'] = catColumns.ExterQual.map(ExterQual_dict)

In [ ]:
LotShape_dict = {'Reg': 4,
                      'IR1': 3,
                      'IR2': 2,
                     'IR3':1
                    }
catColumns['LotShape'] = catColumns.LotShape.map(LotShape_dict)

In [ ]:
Utilities_dict = {'AllPub': 4,
                      'NoSewr': 3,
                      'NoSewa': 2,
                     'ELO':1
                    }
catColumns['Utilities'] = catColumns.Utilities.map(Utilities_dict)

In [ ]:
ExterQual_dict = {'Ex':5,
                  'Gd': 4,
                      'TA': 3,
                      'Fa': 2,
                     'Po':1
                    }
catColumns['ExterQual'] = catColumns.ExterQual.map(ExterQual_dict)

In [ ]:
ExterCond_dict = {'Ex':5,
                  'Gd': 4,
                      'TA': 3,
                      'Fa': 2,
                     'Po':1
                    }
catColumns['ExterCond'] = catColumns.ExterCond.map(ExterCond_dict)

In [ ]:
BsmtQual_dict = {'Ex':5,
                  'Gd': 4,
                      'TA': 3,
                      'Fa': 2,
                     'Po':1,
                  'NA':0
                    }
catColumns['BsmtQual'] = catColumns.BsmtQual.map(BsmtQual_dict)

In [ ]:
BsmtCond_dict = {'Ex':5,
                  'Gd': 4,
                      'TA': 3,
                      'Fa': 2,
                     'Po':1,
                  'NA':0
                    }
catColumns['BsmtCond'] = catColumns.BsmtCond.map(BsmtCond_dict)

In [ ]:
BsmtExposure_dict = {'Gd': 4,
                      'Av': 3,
                      'Mn': 2,
                     'No':1,
                  'NA':0
                    }
catColumns['BsmtExposure'] = catColumns.BsmtExposure.map(BsmtExposure_dict)

In [ ]:
BsmtFinType1_dict = {'GLQ':6,
                 'ALQ':5,
                  'BLQ': 4,
                      'Rec': 3,
                      'LwQ': 2,
                     'Unf':1,
                  'NA':0
                    }
catColumns['BsmtFinType1'] = catColumns.BsmtFinType1.map(BsmtFinType1_dict)

In [ ]:
BsmtFinType2_dict = {'GLQ':6,
                 'ALQ':5,
                  'BLQ': 4,
                      'Rec': 3,
                      'LwQ': 2,
                     'Unf':1,
                  'NA':0
                    }
catColumns['BsmtFinType2'] = catColumns.BsmtFinType2.map(BsmtFinType2_dict)

In [ ]:
HeatingQC_dict = {'Ex':5,
                  'Gd': 4,
                      'TA': 3,
                      'Fa': 2,
                     'Po':1
                    }
catColumns['HeatingQC'] = catColumns.HeatingQC.map(HeatingQC_dict)

In [ ]:
Electrical_dict = {'SBrkr':5,
                  'FuseA': 4,
                      'FuseF': 3,
                      'FuseP': 2,
                     'Mix':1
                    }
catColumns['Electrical'] = catColumns.Electrical.map(Electrical_dict)

In [ ]:
KitchenQual_dict = {'Ex':5,
                  'Gd': 4,
                      'TA': 3,
                      'Fa': 2,
                     'Po':1
                    }
catColumns['KitchenQual'] = catColumns.KitchenQual.map(KitchenQual_dict)

In [ ]:
Functional_dict = {'Typ':7,
                   'Min1':6,
                 'Min2':5,
                  'Mod': 4,
                      'Maj1': 3,
                      'Maj2': 2,
                     'Sev':1,
                  'Sal':0
                    }
catColumns['Functional'] = catColumns.Functional.map(Functional_dict)

In [ ]:
GarageFinish_dict = {'Fin': 3,
                      'RFn': 2,
                      'Unf': 1,
                     'NA':0
                    }
catColumns['GarageFinish'] = catColumns.GarageFinish.map(GarageFinish_dict)

In [ ]:
GarageQual_dict = {'Ex':5,
                  'Gd': 4,
                      'TA': 3,
                      'Fa': 2,
                     'Po':1,
                   'NA':0
                    }
catColumns['GarageQual'] = catColumns.GarageQual.map(GarageQual_dict)

In [ ]:
GarageCond_dict = {'Ex':5,
                  'Gd': 4,
                      'TA': 3,
                      'Fa': 2,
                     'Po':1,
                   'NA':0
                    }
catColumns['GarageCond'] = catColumns.GarageCond.map(GarageCond_dict)

In [ ]:
PavedDrive_dict = {'Y': 3,
                      'P': 2,
                     'N':1
                    }
catColumns['PavedDrive'] = catColumns.PavedDrive.map(PavedDrive_dict)

Since the number of missing values in the categorical variables is not large as per the criteria, missing values were imputed using the mode of the category.

In [ ]:
for var in list(catColumns.columns):
    if catColumns[var].isnull().mean() > 0:
        catColumns = catColumns.fillna(catColumns[var].value_counts().index[0])

In [ ]:
print(catColumns.isnull().mean() > 0) # no more missing values

### Handling numeric columns


In [ ]:
numColumns = data._get_numeric_data()

Proportion of missing data in numeric columns

In [ ]:
numColumns.isna().sum()

LotFrontage and GarageYrBlt have the highest number of missing values. 

LotFrontage, MasVnrArea, BsmtFinSF1, BsmtFinSF2, BsmtUnfSF and GarageArea will be imputed using the median or mean

GarageCars, BsmtFullBath, BsmtHalfBath will be imputed from most commonly occuring value

For missing values in GarageYrBuilt, these indicate absence of a garage and will be set to 0


In [ ]:
numColumns.GarageYrBlt.fillna(0, inplace=True)

In [ ]:
# filling with most commonly occuring value

discrete = ['BsmtFullBath', 'BsmtHalfBath', 'GarageCars']
for var in discrete:
    if numColumns[var].isnull().mean() > 0:
        numColumns = numColumns.fillna(numColumns[var].value_counts().index[0])

In [ ]:
# filling with the median
numColumns.LotFrontage = data.groupby('Neighborhood')['LotFrontage'].\
                    transform(lambda x: x.fillna(x.median()))

In [ ]:
numColumns.MasVnrArea = data.groupby('Neighborhood')['MasVnrArea'].\
                            transform(lambda x: x.fillna(x.median()))

numColumns.BsmtFinSF1 = data.groupby('Neighborhood')['BsmtFinSF1'].\
                            transform(lambda x: x.fillna(x.median()))

numColumns.BsmtFinSF2 = data.groupby('Neighborhood')['BsmtFinSF2'].\
                            transform(lambda x: x.fillna(x.median()))

numColumns.BsmtUnfSF = data.groupby('Neighborhood')['BsmtUnfSF'].\
                            transform(lambda x: x.fillna(x.median()))

In [ ]:
numColumns.isna().mean() # more missing values

### Exploratory data analysis on numeric columns

In [ ]:
# looking at time-related variables (YearBuilt, YearRemodAdd, GarageYrBlt, MoSold, YrSold)

# When are houses sold?
data.groupby(['YrSold','MoSold']).Id.count().plot(kind='bar', figsize=(14,4))
plt.xlabel("Year & Month")
plt.ylabel("Number of houses")
plt.title('When are houses sold?')

plt.show()

Seasonal pattern for house sales. House sales peak mid-year each year.

In [ ]:
# When were garages built?
data.groupby(['GarageYrBlt']).Id.count().plot(kind='bar', figsize=(14,4))
plt.ylabel("Number of garages built")
plt.xlabel("Year")
plt.title('When garages were built?')
plt.show()

Garages became more common in the 2000's

In [ ]:
# When houses were built
data.groupby(['YearBuilt']).Id.count().plot(kind='bar', figsize=(14,4))
plt.title('When houses were built')
plt.ylabel("Number of houses built")
plt.xlabel("Year")
plt.show()

In [ ]:
# When houses were remodelled
data.groupby(['YearRemodAdd']).Id.count().plot(kind='bar', figsize=(14,4))
plt.title('When houses were remodeled')
plt.ylabel("Number of houses")
plt.xlabel("Year of remodelling")
plt.show()

Most houses remodelled in the 50's

In [ ]:
# Where are the houses?
data.groupby('Neighborhood').Id.count().\
    sort_values().\
    plot(kind='barh', figsize=(6,6))
plt.title('What neighborhoods are houses in?')
plt.xlabel("Number of houses")
plt.show()

## Generating new features

All are numeric. MoSold will be factorised since the euclidean distance between the numbers doesn't offer a lot of information

Difference between YearBuilt and YrSold is the age of the house when it was sold - HouseAge

In [ ]:
# concatenating numColumns and catColumns
dataFull = pd.concat([numColumns, catColumns], axis=1, sort=False)

In [ ]:
# checking that dataFull meets the expected dimensions and no missing values
dataFull.shape
dataFull.isna().sum().mean()

In [ ]:
dataFull.head()

Similar pattern to building of garages. More houses built with garages

Related features that can be summed up

TotalBath = BsmtFullBath + BsmtHalfBath + FullBath + HalfBAth

TotalFlrSF = 1stFlrSF + 2ndFlrSF + LowQualFinSF

TotalBsmtSF is assumed to relate BsmtFinSF1 + BsmtFinSF2 + BsmtUnfSF 

TotalRooms - TotRmsAbvGrd + TotalBath

HouseSF = GarageArea + WoodDeckSF + TotalBsmtSF + GrLivArea

TotalPorchSF = OpenPorchSF + EnclosedPorch + 3-SsnPorch + ScreenPorch

HouseAgeYr = YrSold - YearBuilt 

In [ ]:
numNew = ['TotalBath', 'TotalFlrSF', 'TotalBsmtSF', 'TotalRooms', 'HouseSF', 'TotalPorchSF', 'HouseAgeYr']

In [ ]:
#total number of baths
dataFull['TotalBath'] = dataFull['BsmtFullBath'] + (dataFull['BsmtHalfBath']*0.5) \
                            + dataFull['FullBath'] + (dataFull['HalfBath']*0.5)

#total floor square feet
dataFull['TotalFlrSF'] = dataFull['1stFlrSF'] + dataFull['2ndFlrSF'] + dataFull['LowQualFinSF']

#total number of rooms
dataFull['TotalRooms'] =  dataFull['TotRmsAbvGrd'] + dataFull['TotalBath']

#total size of the house in SF
dataFull['HouseSF'] = dataFull['GarageArea'] + dataFull['WoodDeckSF'] + dataFull['TotalBsmtSF']\
                    + dataFull['GrLivArea']

#total size of porches
dataFull['TotalPorchSF'] = dataFull['OpenPorchSF'] + dataFull['EnclosedPorch'] +\
                            dataFull['3SsnPorch'] + dataFull['ScreenPorch']

#age of the house
dataFull['HouseAgeYr'] = dataFull['YrSold'] - dataFull['YearBuilt']

Also creating features for factors that are taken into consideration during purchase of a house such as;
- is the house new or remodeled
- does it have a garage
- does it have a basement
- does it have a porch
- does it have a pool

In [ ]:
dataFull['HasBasement'] = dataFull.TotalBsmtSF.apply(lambda x: 1 if x > 0 else 0)
dataFull['HasGarage'] = dataFull.GarageArea.apply(lambda x: 1 if x > 0 else 0)
dataFull['HasPorch'] = dataFull.TotalPorchSF.apply(lambda x: 1 if x > 0 else 0)
dataFull['HasPool'] = dataFull.PoolArea.apply(lambda x: 1 if x > 0 else 0)
dataFull['WasRemodeled'] = (dataFull.YearRemodAdd != dataFull.YearBuilt).astype(np.int64)
dataFull['IsNew'] = (dataFull.YearBuilt > 2000).astype(np.int64)
dataFull['WasCompleted'] = (dataFull.SaleCondition != 'Partial').astype(np.int64)


In [ ]:
catNew = ['HasBasement','HasGarage', 'HasPorch', 'HasPool', 'WasRemodeled', 'IsNew', 'WasCompleted']

In [ ]:
# resplitting the data
dF_train, dF_test = dataFull[0:1460], dataFull[1460:]

In [ ]:
# joining dF_train to the target variable
dF_train = pd.concat([dF_train, pd.Series(y_train)], axis=1, sort=False)

In [ ]:
dF_train.rename(columns={0:'SalePrice'}, inplace=True)

## More exploratory analysis to understand distribution of the data

In [ ]:
dF_train.describe()

In [ ]:
pearsoncorr = dF_train.corr(method='pearson')

# visualizing the correlation matrix as a heatmap
plt.figure(figsize=(60,60))
top_corr = pearsoncorr.index
sns.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdYlGn',
            annot=False,
            linewidth=1.0)

There features that are highly correlated with each other as well as with the target variable. (multicollinearity)

Also, there are features that are highly correlated with SalePrice. These are GrLivArea, OverallQual, TotalBsmtSF, 1stFlrSF, the new features (TotalBath, Total1stFlrSF, TotalRooms and HouseSF), as well as HouseAgeYr (highest negative correlation).

Medium correlated features are LotFrontage,LotArea, YearBuilt, YearRemodAdd, MasVnrArea, BsmtFinSF1, 

In [ ]:
# Feature sorted by correlation to SalePrice

corr = pearsoncorr.sort_values('SalePrice', ascending=False)
plt.figure(figsize=(30,30))
sns.barplot( corr.SalePrice[1:], corr.index[1:], orient='h')
plt.show()

Generated features HouseAgeYr and HouseSF have the highest negative and positive correlation respectively.

Categorical and numerical variables are treated seperately

#### Categorical and discrete features

In [ ]:
discrete = ['MoSold','YearBuilt','YearRemodAdd','YrSold','GarageYrBlt',
                                           'TotalBath', 'Fireplaces', 'BsmtFullBath', 'BsmtHalfBath',
                                           'FullBath', 'HalfBath', 'TotalRooms', 'MSSubClass','GarageCars',
                                           'TotRmsAbvGrd','BedroomAbvGr', 'HouseAgeYr', 'OverallCond','KitchenAbvGr']
catFeatures = catNew + nominal + ordinal + discrete
numFeatures = num_cols + numNew

In [ ]:
numFeatures = list(set(numFeatures) - set(discrete))

In [ ]:
total = catFeatures + numFeatures
df = list(dF_train.columns)

list(set(df) - set(total))

In [ ]:
# Count plots of categorical & discrete features

f = pd.melt(dF_train, value_vars=sorted(catFeatures)) # similar to stack() function in R
g = sns.FacetGrid(f, col='variable', col_wrap=4, sharex=False, sharey=False)
plt.xticks(rotation='vertical')
g = g.map(sns.countplot, 'value')
[plt.setp(ax.get_xticklabels(), rotation=60) for ax in g.axes.flat]
g.fig.tight_layout()
plt.show()

In [ ]:
# Count plots of categorical features
f = pd.melt(dF_train, id_vars=['SalePrice'], value_vars=sorted(catFeatures))
g = sns.FacetGrid(f, col='variable', col_wrap=3, sharex=False, sharey=False, size=4)
g = g.map(sns.boxplot, 'value', 'SalePrice')
[plt.setp(ax.get_xticklabels(), rotation=90) for ax in g.axes.flat]
g.fig.tight_layout()
plt.show()

From the box plots, it can be seen that variables have outliers on the upper end of the SalePrice above 700,000.

In [ ]:
# which categorical feature contributes the most to predicting the SalePrice
import scipy.stats

anova = {'feature':[], 'f':[], 'p':[]}
for cat in catFeatures:
    group_prices = []
    for group in dF_train[cat].unique():
        group_prices.append(dF_train[dF_train[cat] == group]['SalePrice'].values)
    f, p = scipy.stats.f_oneway(*group_prices)
    anova['feature'].append(cat)
    anova['f'].append(f)
    anova['p'].append(p)
anova = pd.DataFrame(anova)
anova = anova[['feature','f','p']]
anova.sort_values('p', inplace=True)

# Plot
plt.figure(figsize=(14,6))
sns.barplot(anova.feature, np.log(1./anova['p']))
plt.xticks(rotation=90)
plt.show()

#### Continuous features

In [ ]:
numFeatures = list(set(numFeatures) - set(['Id']))

In [ ]:
# Grid of distribution plots of all numerical features
# f = pd.melt(dF_train, value_vars=sorted(numFeatures))
# g = sns.FacetGrid(f, col='variable', col_wrap=4, sharex=False, sharey=False)
# g = g.map(sns.distplot, 'value')

dF_train[numFeatures].plot(kind='density', subplots=True, layout=(6,4), figsize=(30,30))

In [ ]:
# Scatter plots of numerical features against SalePrice
f = pd.melt(dF_train, id_vars=['SalePrice'], value_vars=sorted(numFeatures))
g = sns.FacetGrid(f, col='variable', col_wrap=4, sharex=False, sharey=False)
plt.xticks(rotation='vertical')
g = g.map(sns.regplot, 'value', 'SalePrice', scatter_kws={'alpha':0.3})
[plt.setp(ax.get_xticklabels(), rotation=60) for ax in g.axes.flat]
g.fig.tight_layout()
plt.show()


### Feature transformations

Log transformation of right skewed features

Rescaling

## Multivariate regression models <br/>
Since the number of features is very large, regression models wih regularisation will be used to avoid overfitting and making complex models.

Models to be used:
- Ridge regression
- LASSO regression

These will be explored later:
- Kernel ridge regression
- Stochastic gradient descent
- ElasticNet

In [ ]:
# dummy variables for nominal and categorical variables

# cat_dum = ['MSSubClass','OverallQual',
#        'OverallCond','MSZoning', 'Street', 'LotShape',
#        'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
#        'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
#        'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
#        'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
#        'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
#        'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish',
#        'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']

model_data = pd.get_dummies(dataFull)

# resplitting
model_train, model_test = model_data[0:1460], model_data[1460:]

In [ ]:
# remove anything over 4,000 sq ft og GrLivArea
# model_train.drop(dF_train[dF_train.GrLivArea >= 4000].index, inplace=True)

In [ ]:
# scaling numerical features

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model_train.loc[:,numFeatures] = scaler.fit_transform(model_train[numFeatures])
model_test.loc[:,numFeatures] = scaler.fit_transform(model_test[numFeatures])

In [ ]:
# splitting data into model_train into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = \
                    train_test_split(model_train, y_train, 
                                     test_size=0.3, random_state=42)

### Linear regression with cross validation

In [ ]:
# multiple linear regression - least squares fitting
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
kfold = KFold(n_splits = 10, random_state = 42)
lm = LinearRegression()

# fitting the model
lm.fit(model_train, ytrain_log)
mse = cross_val_score(lm, model_train, y_train, scoring="neg_mean_squared_error",
                     cv=10)
mean_mse = np.mean(mse)

In [ ]:
from sklearn import metrics

# Make cross validated predictions
lm_predictions = cross_val_predict(lm, model_train, y_train, cv=10)

plt.scatter(y_train, lm_predictions)


accuracy = metrics.r2_score(y_train, lm_predictions)
print('Cross-Predicted Accuracy:', accuracy)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
#predicting on new data and writing predictions to a file
y_predictions = lm.predict(model_test)
ypred_df = pd.concat([df_test.Id, pd.Series(y_predictions)], axis=1)
ypred_df.columns = ['Id','SalePrice']

ypred_df.to_csv('2d0.csv', index=False)

### Ridge regression with regularisation

In [ ]:
# ridge regression with cross validation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
ridge = Ridge()

parameters = {'alpha':[1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}

ridge_regressor = GridSearchCV(ridge, parameters, scoring="neg_mean_squared_error",
                              cv=10)

ridge_regressor.fit(model_train, y_train)

In [ ]:
print(ridge_regressor.best_params_)

In [ ]:
print(ridge_regressor.best_score_)

In [ ]:
# Make cross validated predictions
r_predictions = ridge_regressor.predict(model_train,)

plt.scatter(y_train, r_predictions)


accuracy = metrics.r2_score(y_train, r_predictions)
print('Cross-Predicted Accuracy:', accuracy)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
#predicting on new data and writing predictions to a file
ridge_predictions = ridge_regressor.predict(model_test)

ridgepred_df = pd.concat([df_test.Id, pd.Series(ridge_predictions)], axis=1)
ridgepred_df.columns = ['Id','SalePrice']

ridgepred_df.to_csv('2d1.csv', index=False)

### Lasso regression

In [ ]:
# lasso regression
from sklearn.linear_model import Lasso
lasso = Lasso()
parameters = {'alpha':[1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring="neg_mean_squared_error",
                              cv=10)


lasso_regressor.fit(model_train, y_train)

In [ ]:
print(lasso_regressor.best_score_)
print(lasso_regressor.best_estimator_)

In [ ]:
# Make cross validated predictions
l_predictions = lasso_regressor.predict(model_train)

plt.scatter(y_train, l_predictions)


accuracy = metrics.r2_score(y_train, l_predictions)
print('Cross-Predicted Accuracy:', accuracy)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
#predicting on new data and writing predictions to a file
lasso_predictions = lasso_regressor.predict(model_test)

lassopred_df = pd.concat([df_test.Id, pd.Series(lasso_predictions)], axis=1)
lassopred_df.columns = ['Id','SalePrice']

lassopred_df.to_csv('2d2.csv', index=False)

### Kernel ridge regression

In [ ]:
from sklearn.kernel_ridge import KernelRidge

kernel = KernelRidge()
parameters = {'alpha':[1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}

kernel_regressor = GridSearchCV(kernel, parameters, scoring="neg_mean_squared_error",
                              cv=10)


kernel_regressor.fit(model_train, y_train)

#predicting on new data and writing predictions to a file
kernel_predictions = kernel_regressor.predict(model_test)

kernelpred_df = pd.concat([df_test.Id, pd.Series(kernel_predictions)], axis=1)
kernelpred_df.columns = ['Id','SalePrice']

kernelpred_df.to_csv('2d3.csv', index=False)

In [ ]:
# Make cross validated predictions
kr_predictions = kernel_regressor.predict(model_train,)

plt.scatter(y_train, kr_predictions)


accuracy = metrics.r2_score(y_train, kr_predictions)
print('Cross-Predicted Accuracy:', accuracy)

plt.xticks(())
plt.yticks(())

plt.show()

### Stochastic Gradient Descent Regressor


In [ ]:
from sklearn.linear_model import SGDRegressor

sgd = SGDRegressor(max_iter=1000, tol=1e-3)

# fitting the model
kfold = KFold(n_splits = 10, random_state = 42)
sgd.fit(model_train, ytrain_log)
mse = cross_val_score(sgd, model_train, y_train, scoring="neg_mean_squared_error",
                     cv=10)
mean_mse = np.mean(mse)


#predicting on new data and writing predictions to a file
sgd_predictions = sgd.predict(model_test)
sgdpred_df = pd.concat([pd.DataFrame(df_test.Id), 
                        pd.DataFrame(sgd_predictions)], axis=1)
sgdpred_df.columns = ['Id','SalePrice']

sgdpred_df.to_csv('2d4.csv', index=False)

In [ ]:
# Make cross validated predictions
sgd_predictions = cross_val_predict(sgd, model_train, y_train, cv=10)

plt.scatter(y_train, sgd_predictions)


accuracy = metrics.r2_score(y_train, sgd_predictions)
print('Cross-Predicted Accuracy:', accuracy)

plt.xticks(())
plt.yticks(())

plt.show()

### ElasticNet with cross validation

In [ ]:
from sklearn.linear_model import ElasticNetCV
elastic = ElasticNetCV(cv=10, random_state=42)

elastic.fit(model_train, y_train)

#predicting on new data and writing predictions to a file
elastic_predictions = elastic.predict(model_test)
elasticpred_df = pd.concat([pd.DataFrame(df_test.Id), 
                            pd.DataFrame(elastic_predictions)], axis=1)
elasticpred_df.columns = ['Id','SalePrice']

#elasticpred_df.set_index(elasticpred_df.iloc[:,0])

elasticpred_df.to_csv('2d5.csv', index=False)

In [ ]:
# Make cross validated predictions
el_predictions = elastic.predict(model_train)

plt.scatter(y_train, el_predictions)


accuracy = metrics.r2_score(y_train, el_predictions)
print('Cross-Predicted Accuracy:', accuracy)

plt.xticks(())
plt.yticks(())

plt.show()

## Deep learning techinques

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# # detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# # instantiating the model in the strategy scope creates the model on the TPU

with tpu_strategy.scope():
    
    model = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(88,), kernel_initializer='normal'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation = 'linear')])

    optimizer = tf.keras.optimizers.RMSprop(0.001)
    
    
    # for mean squared error regression problem
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])

In [ ]:
model.summary()

In [ ]:
# training the model
EPOCHS = 10
history = model.fit(X_train, Y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=1,
  callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
# visualising training process
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('Sale price')

In [ ]:
plotter.plot({'Basic': history}, metric = "mse")
plt.ylim([0, 20])
plt.ylabel('MSE [sale price^2]')

In [ ]:
# making predictions
Y_predictions = model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(Y_test, Y_predictions)
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
# error distribution - expect normal distribution
error = Y_predictions - Y_test
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [sale price]")
_ = plt.ylabel("Count")